## Lab 1: Creating a Customer Service Assistant prototype

### Overview

In this first lab, we'll build a Customer Service Assistant prototype that will evolve throughout the workshop into a production-ready system serving multiple customers with persistent memory, shared tools, and full observability. Our agent will have the following three core tools available:
- **get_return_policy()** - Get return policy information from dedicated knowledge base
- **get_product_info()** - Get product specifications from dedicated knowledge base
- **track_order()** - Track order status using various identifiers

**Workshop Journey:**
- **Lab 1 (Current)**: Create Agent Prototype - Build a functional Customer Service Assistant
- **Lab 2**: Enhance with Memory - Add conversation context and personalization
- **Lab 3**: Scale with Gateway & Identity - Share tools across agents securely
- **Lab 4**: Deploy to Production - Use AgentCore Runtime with observability
- **Lab 5**: Build User Interface - Create a customer-facing application


### Architecture for Lab 1
<div style="text-align:left">
    <img src="images/Intelligent Customer Service Assistant Lab1 - Strands.png" width="75%"/>
</div>

*Simple prototype running locally. In subsequent labs, we'll migrate this to AgentCore services with shared tools, persistent memory, and production-grade observability.*

### Step 1: Install Dependencies and Import Libraries
Before we start, let's install the pre-requisites for this lab

In [1]:
# Install required packages
%pip install -U -r requirements.txt -q --no-deps
print("[WHITE HEAVY CHECK MARK] requirements.txt installed")

Note: you may need to restart the kernel to use updated packages.
[WHITE HEAVY CHECK MARK] requirements.txt installed


We can now import the required libraries and initialize our boto3 session

In [2]:
# Import libraries
import boto3
from boto3.session import Session


from strands.tools import tool
from strands_tools import retrieve
from lab_helpers.utils import put_ssm_parameter

print("[WHITE HEAVY CHECK MARK] Required libraries are imported")

[WHITE HEAVY CHECK MARK] Required libraries are imported


In [3]:
# Get boto session
boto_session = Session()
region = boto_session.region_name
print("[WHITE HEAVY CHECK MARK] Boto3 session initialized")

[WHITE HEAVY CHECK MARK] Boto3 session initialized


### Step 2: Knowledge Base Setup

The Customer Service Assistant uses two dedicated knowledge bases that are automatically set up by the infrastructure stack:

- **Return Policy Knowledge Base**: Contains return policy information and procedures
- **Product Information Knowledge Base**: Contains product specifications and details

These knowledge bases are created and populated with sample data during the infrastructure deployment. The knowledge base IDs are stored in AWS Systems Manager Parameter Store and automatically retrieved by the tools when needed.

**Note**: In a production environment, you would populate these knowledge bases with your actual return policies and product catalogs.

### Step 3: Implementing custom tools

Next, we will implement the three tools which will be provided to the Customer Service Assistant.

Defining tools in Strands Agent is extremely simple, just add a `@tool` decorator to your function, and provide a description of the tool in the function's docstring. Strands Agents will use the function documentation, typing and arguments to provide context on this tool to your agent. 


#### Tool 1: Get Return Policy

**Purpose:** This tool helps customers understand return policies for different product categories. It provides detailed information about return windows, conditions, processes, and refund timelines so customers know exactly what to expect when returning items. This tool uses the return policy knowledge base to get the information. 

In [4]:
@tool
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information from the Return Policy Knowledge Base.
    
    Args:
        product_category: Product category (e.g., 'smartphones', 'laptops', 'accessories')
    
    Returns:
        Formatted return policy information from knowledge base
    """
    try:
        # Get Return Policy KB ID from parameter store
        ssm = boto3.client('ssm')
        account_id = boto3.client('sts').get_caller_identity()['Account']
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name='/app/customersupport/kb/return-policy-kb-id')['Parameter']['Value']
        print(f"Successfully retrieved Return Policy KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "return_policy_query",
            "input": {
                "text": f"return policy for {product_category}",
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4
            }
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access return policy information. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in get_return_policy: {str(e)}")
        return f"Unable to access return policy information. Error: {str(e)}"

print("[WHITE HEAVY CHECK MARK] Return policy tool ready")

[WHITE HEAVY CHECK MARK] Return policy tool ready


#### Tool 2: Get Product Information

**Purpose:** This tool provides customers with comprehensive product details including warranties, available models, key features, shipping policies, and return information. It helps customers make informed purchasing decisions and understand what they're buying. This tool uses the product information knowledge base to retrieve the information.

In [5]:
@tool
def get_product_info(product_type: str) -> str:
    """
    Get product information from the Product Information Knowledge Base.
    
    Args:
        product_type: Product type (e.g., 'smartphones', 'laptops', 'headphones', 'monitors')
    
    Returns:
        Formatted product information from knowledge base
    """
    try:
        # Get Product Information KB ID from parameter store
        ssm = boto3.client('ssm')
        account_id = boto3.client('sts').get_caller_identity()['Account']
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name='/app/customersupport/kb/product-info-kb-id')['Parameter']['Value']
        print(f"Successfully retrieved Product Information KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "product_info_query",
            "input": {
                "text": f"product information specifications for {product_type}",
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4
            }
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access product information. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in get_product_info: {str(e)}")
        return f"Unable to access product information. Error: {str(e)}"

print("[WHITE HEAVY CHECK MARK] Product information tool ready")

[WHITE HEAVY CHECK MARK] Product information tool ready


#### Tool 3: Order Tracking

**Purpose:** This tool allows customers to track their orders using various identifiers or natural language queries.

In [6]:
@tool
def track_order(order_id: str = None, tracking_id: str = None, customer_id: str = None, natural_query: str = None) -> str:
    """Track order status using order ID, tracking ID, customer ID, or natural language query.
    
    Args:
        order_id: Order ID (e.g., 'ORD-2025-001234')
        tracking_id: Shipping tracking ID (e.g., '1Z999AA1234567890')
        customer_id: Customer ID (e.g., 'CUST001')
        natural_query: Natural language query (e.g., 'my iPhone order from last Friday')
    
    Returns:
        Order status and tracking information
    """
    # Mock order database - in real implementation, this would query DynamoDB
    orders = {
        "ORD-2025-001234": {
            "customer_id": "CUST001",
            "tracking_id": "1Z999AA1234567890",
            "product_name": "iPhone 17 Pro 256GB",
            "order_date": "2025-12-02",
            "status": "shipped",
            "estimated_delivery": "2025-12-12"
        },
        "ORD-2024-001235": {
            "customer_id": "CUST002",
            "tracking_id": "1Z999BB5678901234",
            "product_name": "MacBook Pro 14-inch",
            "order_date": "2024-01-10",
            "status": "delivered",
            "estimated_delivery": "2024-01-15"
        }
    }
    
    # Find order by different identifiers
    found_order = None
    found_order_id = None
    
    if order_id:
        found_order = orders.get(order_id)
        found_order_id = order_id
    elif tracking_id:
        for oid, order in orders.items():
            if order.get("tracking_id") == tracking_id:
                found_order = order
                found_order_id = oid
                break
    elif customer_id:
        customer_orders = [(oid, order) for oid, order in orders.items() if order.get("customer_id") == customer_id]
        if len(customer_orders) == 1:
            found_order_id, found_order = customer_orders[0]
        elif len(customer_orders) > 1:
            return f"Multiple orders found for customer {customer_id}. Please provide order ID or tracking ID for specific order."
    
    if found_order:
        return f"Order Status - {found_order_id}:\n\n" \
               f"• Product: {found_order['product_name']}\n" \
               f"• Order Date: {found_order['order_date']}\n" \
               f"• Status: {found_order['status'].title()}\n" \
               f"• Tracking ID: {found_order['tracking_id']}\n" \
               f"• Estimated Delivery: {found_order['estimated_delivery']}"
    else:
        return "Order not found. Please check your order ID, tracking ID, or contact customer service for assistance."

print("[WHITE HEAVY CHECK MARK] Order tracking tool ready")

[WHITE HEAVY CHECK MARK] Order tracking tool ready


### Step 4: Create and Configure the Customer Service Assistant

Next, we will create the Customer Service Assistant providing a model, the list of tools implemented in the previous step, and with a system prompt. The system prompt provides general instructions and desired behavior for the agent.

In [7]:
from strands import Agent
from strands.models import BedrockModel

SYSTEM_PROMPT = """You are a helpful and professional Customer Service Assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Help customers with return policies, product information, and order tracking
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following three core tools:
1. get_return_policy() - Get return policy information from our dedicated knowledge base
2. get_product_info() - Get product specifications and information from our dedicated knowledge base
3. track_order() - Track order status using order ID, tracking ID, customer ID, or natural language queries

Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about products, policies, or order status.If you found the right tool, use it"""

# Initialize the Bedrock model (Anthropic Claude 4.5 Haiku)
model = BedrockModel(
    model_id="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=region
)

# Create the Customer Service Assistant with three core tools
agent = Agent(
    model=model,
    tools=[
        get_return_policy,  # Tool 1: Return policy information from knowledge base
        get_product_info,   # Tool 2: Product information from knowledge base
        track_order         # Tool 3: Order tracking functionality
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("[WHITE HEAVY CHECK MARK] Customer Service Assistant created successfully!")

[WHITE HEAVY CHECK MARK] Customer Service Assistant created successfully!


### Step 5: Test the Customer Service Assistant

Let's test our agent with sample queries to ensure all three core tools work correctly.

#### Test Return Policy Query

In [8]:
response = agent("What's the return policy for laptops?")


Tool #1: get_return_policy
Successfully retrieved Return Policy KB ID: 27Y8AQDR8V
Great question! Here's our return policy for laptops:

## **Laptop Return Policy**

**Return Window:**
- 30-day return window applies to laptops

**Requirements:**
- Item must be in original condition with all included components
- Proof of purchase is required

**Return Process:**
1. Contact customer service or use our online return portal
2. Receive a return authorization number (RMA)
3. Package the laptop securely with all components
4. Ship using the provided prepaid label or approved carrier
5. Track your return status through our online portal

**Return Shipping:**
- **Free return shipping** for defective items
- **Free return shipping** for orders over $50 (change of mind returns)
- Customers pay shipping for change of mind returns under $50
- Express return processing is available for an additional fee

**Inspection & Refund Timeline:**
- All returns are inspected upon receipt
- Functional testin

In [9]:
response = agent("Can you tell me about your smartphone specifications?")


Tool #2: get_product_info
Successfully retrieved Product Information KB ID: FVWRG1ZTCH
I was able to retrieve some general product information about our electronics catalog, which includes smartphones. However, the detailed smartphone specifications aren't fully showing in the results. 

Here's what I can tell you about our smartphone offerings:

**General Smartphone Information:**
- We carry multiple brands and price points to meet diverse customer needs
- Smartphones come with standard manufacturer warranties (typically 1-3 years)
- Extended warranty and accidental damage protection plans are available
- We offer financing options and payment plans
- Trade-in credit programs are available for device upgrades

**Support & Compatibility:**
- Free technical support during warranty period
- Phone, chat, and email support channels available
- Video troubleshooting sessions for complex issues
- Carrier compatibility information provided
- Regional variations in features and connectivity



#### Test Order Tracking

In [ ]:
response = agent("What's the status of order ORD-2025-001234?")

## [PARTY POPPER] Lab 1 Complete!

You've successfully created a functional Customer Service Assistant prototype! Here's what you accomplished:

- Built an agent with 3 core tools (return policy, product information, order tracking)  
- Integrated dual knowledge bases for return policies and product information  
- Tested customer service scenarios with knowledge base queries and order tracking  
- Established the foundation for our production journey  

### Current Limitations (We'll fix these!)
- **Single user conversation memory** - local conversation session, multiple customers need multiple sessions.
- **Conversation history limited to session** - no long term memory or cross session information is available in the conversation.
- **Tools reusability** - tools aren't reusable across different agents  
- **Running locally only** - not scalable
- **Identity** - No user and/or agent identity or access control
- **Observability** - Limited observability into agent behavior
- **Existing APIs** - No access to existing enterprise APIs for customer data

##### Next Up [Lab 2: Personalize our agent by adding memory →](lab-02-agentcore-memory.ipynb)
